<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap12/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 14: Colab Experiment on Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.



In [19]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [20]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [21]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [22]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
  # TODO -- compute the keys, queries and values.
  # Replace these three lines
  # query = np.ones_like(x)
  # key = np.ones_like(x)
  # value = np.ones_like(x)

  query = beta_q + np.dot(omega_q, x)
  key = beta_k + np.dot(omega_k, x)
  value = beta_v + np.dot(omega_v, x)

  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [23]:
def softmax(items_in):

  # TODO Compute the elements of items_out
  # Replace this line
  # items_out = items_in.copy()

  exp_items_in = np.exp(items_in)
  items_out = exp_items_in / np.sum(exp_items_in)

  return items_out

Now compute the self attention values:

In [24]:
# Create emptymlist for output
all_x_prime = []

# For each output
for n in range(N):
  # Create list for dot products of query N with all keys
  all_km_qn = []
  # Compute the dot products
  for key in all_keys:
    # TODO -- compute the appropriate dot product
    # Replace this line
    # dot_product = 1
    dot_product = np.dot(key.transpose(), all_queries[n])


    # Store dot product
    all_km_qn.append(dot_product)

  # Compute dot product
  attention = softmax(all_km_qn)
  # Print result (should be positive sum to one)
  print("Attentions for output ", n)
  print(attention)

  # TODO: Compute a weighted sum of all of the values according to the attention
  # (equation 12.3)
  # Replace this line
  # x_prime = np.zeros((D,1))

  x_prime = np.zeros((D,1))
  for att, value in zip(attention, all_values):
    x_prime += att * value


  all_x_prime.append(x_prime)


# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")


Attentions for output  0
[[[1.24326146e-13]]

 [[9.98281489e-01]]

 [[1.71851130e-03]]]
Attentions for output  1
[[[2.79525306e-12]]

 [[5.85506360e-03]]

 [[9.94144936e-01]]]
Attentions for output  2
[[[0.00505708]]

 [[0.00654776]]

 [[0.98839516]]]
x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [25]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [26]:
 # Now let's compute self attention in matrix form
def self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  # 2. Compute dot products
  # 3. Apply softmax to calculate attentions
  # 4. Weight values by attentions
  # Replace this line
  # X_prime = np.zeros_like(X);

  v = np.dot(omega_v, X) + beta_v
  q = np.dot(omega_q, X) + beta_q
  k = np.dot(omega_k, X) + beta_k

  dot_products = np.matmul(k.transpose(), q)
  attentions = softmax_cols(dot_products)
  X_prime = np.matmul(v, attentions)

  return X_prime

In [27]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.94744244  1.64201168  1.61949281]
 [-0.24348429 -0.08470004 -0.06641533]
 [-0.91310441  4.02764044  3.96863308]
 [-0.44522983  2.18690791  2.15858316]]


### Conclusion and Discussion

The purpose of this task is to build a self-attention mechanism from scratch, demonstrating how to process a set of input vectors to produce new output vectors that capture the relationships between the inputs.

1. We generate random input vectors and initialize weights and biases for the query, key, and value transformations.
2. To compute Queries, Keys, and Values, we apply linear transformations to the inputs using the weights and biases to obtain the query, key, and value vectors for each input based on the equations below. For each query, we compute the dot product with all keys to measure the relevance between inputs.
    - $\mathbf{v}_m=\beta_\mathbf{v}+\Omega_{\mathbf{v}}\mathbf{x}_m$
    - $\mathbf{q}_m=\beta_q+\Omega_{\mathbf{q}}\mathbf{x}_n$
    - $\mathbf{k}_m=\beta_k+\Omega_{\mathbf{k}}\mathbf{x}_m$
3. Normalize the attention scores using the softmax function to obtain attention weights that sum to one, and the equation is shown below.
    - $\alpha=[\mathbf{x}_m,\mathbf{x}_n]=\text{softmax}_m[k_{\bullet}^{T}q_n]=\frac{\exp[k_m^Tq_n]}{\sum_{m^{'}=1}^{N}\exp[k_m^Tq_n]}$
4. Multiply the attention weights with the corresponding value vectors and sum them to produce the output vectors.
    - $sa_n=[\mathbf{x}_1,\cdots,\mathbf{x}_N]=\sum_{m=1}^{N}a=[\mathbf{x}_m,\mathbf{x}_n]\mathbf{v}_m$

Here we re-implement the self-attention mechanism using matrix computations for improved efficiency, followed by executing the matrix-based implementation on the input data to obtain the final output vectors. Then, verify the correctness of the implementation by printing the result. The matrix form of the self attention is shown below.

$$
\begin{align*}
\mathbf{V}[\mathbf{X}]&=\beta_{v}1^T+\Omega_v\mathbf{X} \\
\mathbf{V}[\mathbf{X}]&=\beta_{v}1^T+\Omega_v\mathbf{X} \\
\mathbf{V}[\mathbf{X}]&=\beta_{v}1^T+\Omega_v\mathbf{X} \\
Sa[\mathbf{X}]&=\mathbf{V}[\mathbf{X}]\cdot\text{Softmax}[K[\mathbf{X}]^TQ[\mathbf{X}]]
\end{align*}
$$